# 感知机

## 感知机的作用

​	感知机是一个二分类的线性分类模型，输入为实例的特征向量，输出是+1或-1的一个类别，感知机在特征空间中，将实例划分为正负两类的分离超平面，属于判别模型。


## 感知机的模型

​	输入空间$\mathcal{X}\subset{R^n}$,是一个$n$维向量的集合，输出空间$\mathcal{Y}=\{-1,+1\}$是一个类标记的集合。

输入$x\in\mathcal{X}$表示实例的特征向量，$y\in\mathcal{Y}$表示实例的类别。输入到输出如下函数：
$$
f(x)=sign(wx+b)
$$


​	如果能够将数据集的正实例点和负实例点完全划分到超平面两侧，也就是对所有$y_i=+1$的实例$i$，有$wx_i+b>0$,$y_i=-1$的实例$i$，有$wx_i+b<0$,则这个数据集是个线性可分的，否则就是线性不可分。如图，数据集对应于特征空间中的超平面$S$，使用$wx+b=0$将特征向量分为正负两类

<img src="./配图/感知机.assets/感知机.png" alt="iShot2022-03-03 15.52.27" style="zoom:50%;" />

## 感知机的学习策略

​	首先确保数据集是线性可分的，感知器通过改变$w,b$来寻找到这样的一个超平面，并将损失函数极小化。

​	损失函数计算的是误分类点到超平面的总距离,因为是误分类点，所以$wx_i+b$>0时，$y_i<0$，所以相乘是一个负数，需要在求和前加上一个负号使得损失函数变正:
$$
L(w,b)=-\sum y_i(wx_i+b)
$$

## 感知器算法（原始形式）

​	1.随机选取初$w_0,b_0$

​	2.在训练集中选取数据

​	3.如果这是一个误分类点，即$ y_i(wx_i+b)\leq0$,则进行参数更新：
$$
w = w+\eta y_ix_i\\
b = b+\eta y_i
$$
​	4.重复（2），直至没有误分类点



## 感知器算法（对偶形式）

### 思想：

​	对偶形式的想法就是，将$w$和$b$，标记为实例$x_i$和$y_i$的线性组合，通过求解系数来求得$w$和$b$。当我们逐步修改n次$w$和$b$时，$w$和$b$关于$(x_i,y_i)$的增量可以表示为
$$
w = \sum_{i=1}^N\alpha_iy_ix_i\\
b=\sum_{i=1}^N\alpha_iy_i\\
这里的\alpha_i=\eta_i\eta\geq0
$$
​	实例点的更新次数越多，意味着他距离超平面就越近，也就越难正确分类，这样的实例对学习结果的影响最大，对偶形式也是对算法执行速度的优化。



### 算法：

​	1.随机选取初$\alpha\leftarrow0,b\leftarrow0$

​	2.在训练集中选取数据

​	3.如果这是一个误分类点，即$ y_i(\sum_{j=1}^N\alpha_jy_jx_jx_i+b)\leq0$,则进行参数更新：
$$
\alpha_i=\alpha_i+\eta\\
b = b+\eta y_i
$$
​	4.重复（2），直至没有误分类点

​						因为对偶训练实例仅以内积形式出现，为了方便，可以预先将训练实例的内积算出					并存储为一个矩阵，也就是$Gram$矩阵，每次计算时候调用矩阵节省时间。
$$
G = [x_i\cdot x_j]_{N\times N}
$$

## 神经网络解决方案

试设计一个前馈神经网络来解决XOR问题，要求该前馈神经网络具有 两个隐藏神经元和一个输出神经元，并使用 ReLU 作为激活函数.

In [10]:
import torch
import torch.nn as nn

input = torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])# 四个位置
label = torch.tensor([0,1,1,0]) # 可以看出这样的label情况下，我们并不能简单的用一条直线来分割。

class XOR(nn.Module):
    def __init__(self):
        super(XOR, self).__init__()
        self.hidden1 = nn.Linear(2, 32)
        self.hidden2 = nn.Linear(32, 16)
        self.hidden3 = nn.Linear(16, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.relu(self.hidden3(x))
        return x


model = XOR()
optim = torch.optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()
total_accuracy = 0
for epoch in range(500):
    output = model(input)
    loss = loss_function(output, label)
    optim.zero_grad()
    loss.backward()
    optim.step()
print(output.argmax(1))
print('CE_Loss', loss.item())




tensor([0, 1, 1, 0])
CE_Loss 9.27700602915138e-05
